In [19]:
import time
import sys
# Imports
import base64
import io
import zlib
from datetime import datetime

sys.path.append(".")
from grib_transmitter import email_functions as email_func
from grib_transmitter import saildoc_functions as saildoc_func
from grib_transmitter import inreach_functions as inreach_func
from grib_transmitter import configs

In [ ]:
auth_service = email_func.gmail_authenticate()

In [ ]:
# result = email_func.process_new_inreach_message(auth_service)

previous_messages = email_func._load_previous_messages()
unanswered_messages = email_func._get_new_message_ID(auth_service, previous_messages)

if not unanswered_messages:
    print('No new messages, 1945cf2215de8668 is a good message to use for testing')
    pass

In [12]:
unanswered_messages = '1945cf2215de8668'
message_id = unanswered_messages

In [13]:
grib_path = None
garmin_reply_url = None

In [ ]:
# grib_path, garmin_reply_url = email_func._request_and_process_saildocs_grib(message_id, auth_service)
# print(f"Answered message {message_id}", flush=True)

# return grib_path, garmin_reply_url

msg_text, garmin_reply_url = email_func._fetch_message_text_and_url(message_id, auth_service)

# request saildocs grib data
email_func._send_gmail_message(auth_service, configs.SAILDOCS_EMAIL_QUERY, "", "send " + msg_text)


In [ ]:
time_sent = datetime.utcnow()
from datetime import datetime, timedelta
time_sent += timedelta(minutes=-15)
last_response = saildoc_func.wait_for_saildocs_response(auth_service, time_sent)

In [ ]:
last_response
# saildocs message id: 1945d10126a70f68
# saildocs thread id: threadId=1945cf52222f2a1c

In [ ]:
grib_path = email_func._get_grib_attachment(auth_service, last_response['id'])

In [ ]:
grib_path

# grib to check if picking up here
# /files/attachments\\hrrr20250113002800052.grb' 

In [ ]:
encoded_grib = saildoc_func.encode_saildocs_grib_file(grib_path)

In [ ]:
gribmessage = encoded_grib

In [44]:
# inreach_func.send_messages_to_inreach('8123913208@vtext.com', encoded_grib)

In [45]:
import requests
import random
import time

In [ ]:
message_parts = inreach_func._split_message(gribmessage)

In [52]:
message_str = message_parts[0]

In [ ]:
for message_str in message_parts:
    email_func._send_gmail_message(auth_service, configs.SERVICE_EMAIL, "", "send " + message_str)
    print(message_str)

In [59]:
def decode_grib_file(encoded_data, output_path):
    """
    Decodes a base64 encoded and zlib compressed GRIB file and saves it to a specified location.

    Args:
    - encoded_data (str): The base64 encoded string of the zlib compressed GRIB file content.
    - output_path (str): The path where the decoded GRIB file should be saved.

    Returns:
    - str: Path to the saved file.
    """

    # Split the received data into lines and extract only the message content lines, skipping the order numbers
    lines = encoded_data.strip().split("\n")
    encoded_data_without_nrs = ''.join([lines[i] for i in range(len(lines)) if i % 3 == 1])

    # Decode the base64 encoded data
    zipped_data = base64.b64decode(encoded_data_without_nrs)

    # Decompress the zlib data to obtain the original GRIB file
    grib_binary = zlib.decompress(zipped_data)

    # Save the decoded data to the specified path
    with open(output_path, 'wb') as file:
        file.write(grib_binary)

    return output_path

In [60]:
# Copy all messages received from the inreach
TEXT_RECIEVED = """
0
eJxzD/J0YmCIYGRgkGFiD/7foJjJwCXJyCPOABThYmBgEGUAsRQY/jMwMAHh6peNwacaGHZXNLq48y/gX+DAAAYCHAwMh7zfaHDM82cwYDAHAnc0c5XIM5eH
0
1
gcExS/cBu8amCgawuQBYCiJ1
1
"""

# Define the path where the GRIB file should be saved
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
OUTPUT_PATH = configs.OUTPUT_DIR + f'decoded_grib_{current_time}.grb'

# Use the function to decode and save the GRIB file
df = decode_grib_file(TEXT_RECIEVED, OUTPUT_PATH)

In [ ]:
df